In [61]:
import re
import pandas as pd
from profanity import profanity
import csv
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
import nltk
import spacy
from collections import Counter
from statistics import mean
from profanity_check import predict, predict_prob
from google.oauth2 import service_account
from textblob import TextBlob
from google.cloud import translate_v2 as translate
from google.cloud import storage
nlp = spacy.load('de_core_news_sm')
snowStemmer = SnowballStemmer(language='german')
RAW_DATA_PATH = 'data/raw/'

Helper Functions

In [62]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [63]:
def find_longest_word(word_list):
    longest_word =  max(word_list, key=len)
    return len(longest_word)

In [64]:
def lemmatize_string(input_text):
    doc = nlp(input_text.lower())
    result = ' '.join([x.lemma_ for x in doc])
    doc = nlp(result.title())
    result = ' '.join([x.lemma_ for x in doc]).upper()
    return result

In [65]:
def stemm_List_of_Words(input_text):
    result = [snowStemmer.stem(word).upper() for word in input_text]
    return result

In [66]:
def contains_song(ID, JAHR, MONAT):
    return  len(df.loc[(df['ID'] == ID) & (df['JAHR'] == JAHR) & (df['MONAT'] == MONAT)]) >= 1

In [99]:
def isSeasonal():
    for index, row in df.iterrows():
        if np.isnan(row['SEASONAL?']) or False:
            df.loc[(df['ID'] == row['ID']), 'SEASONAL?'] = contains_song(row['ID'], row['JAHR'] + 1, row['MONAT'])

Data Extraction

In [68]:
df_Lied = pd.read_csv(RAW_DATA_PATH + 'LIED.csv', usecols=['ID','INTERPRET', 'TITEL', 'SPRACHE_DEUTSCH', 'TEXT_TEIL1', 'TEXT_TEIL2', 'TEXT_TEIL3', 'TEXT_TEIL4'])
#print(df_Lied.head())

df_Chart_Position = pd.read_csv(RAW_DATA_PATH + 'CHART_POSITION.csv', usecols=['LIED_ID', 'POSITION', 'DATUM_VON', 'DATUM_BIS'])
#print(df_Chart_Position.head())

#get stopword-list
with open(RAW_DATA_PATH+'Stoppwords.csv', newline='', encoding='UTF-8') as f:
    stopwords_list = list(csv.reader(f))
stopwords_list = [word.upper() for word in flatten(stopwords_list)]

Data Conversion

In [69]:
df_Lied['TEXT'] = df_Lied['TEXT_TEIL1'].fillna('') + df_Lied['TEXT_TEIL2'].fillna('') + df_Lied['TEXT_TEIL3'].fillna('') + df_Lied['TEXT_TEIL4'].fillna('')

df_Chart_Position['DATUM_VON'] = pd.to_datetime(df_Chart_Position['DATUM_VON'])
df_Chart_Position['DATUM_BIS'] = pd.to_datetime(df_Chart_Position['DATUM_BIS'])
df_Chart_Position['DAUER'] = (df_Chart_Position['DATUM_BIS'] - df_Chart_Position['DATUM_VON']).dt.days.astype('int16')
df_Chart_Position['JAHR'] = df_Chart_Position['DATUM_BIS'].dt.year.astype('int16')
df_Chart_Position['MONAT'] =  df_Chart_Position['DATUM_BIS'].dt.month.astype('int16')

Text Preprocessing

In [70]:
df_Lied['processed_TEXT'] = df_Lied['TEXT']

#lemmatization
#df_Lied['processed_TEXT'] = df_Lied.processed_TEXT.apply(lambda text: lemmatize_string(text))

# tokenize
df_Lied['processed_TEXT'] = df_Lied.processed_TEXT.apply(lambda text: nltk.word_tokenize(text))

#Stemming
df_Lied['processed_TEXT'] = df_Lied.processed_TEXT.apply(lambda text: stemm_List_of_Words(text))

#remove stopwords
df_Lied['processed_TEXT'] = df_Lied.processed_TEXT.apply(lambda x: [item for item in x if item not in stopwords_list])

#remove numbers
df_Lied['processed_TEXT'] = df_Lied.processed_TEXT.apply(lambda word_list : [re.sub('\w*\d\w*','', word) for word in word_list])

In [71]:
print(df_Lied['processed_TEXT'][0])

['LASS', 'STRAND', 'WEISST', 'WELCH', 'BRAUCH', 'MAL', 'WIED', 'MEER', 'SAND', 'YEAH', 'BRAUCH', 'MAL', 'WIEDERGRUND', 'BLEIB', 'AI', 'UHR', 'FAELLT', 'BLAU', 'SEH', 'VERSINKT', 'SETZT', 'LANGSAM', 'WASS', 'SPIEGELGLATT', 'BLINKT', 'WOLL', 'SEHNSUCHT', 'MORG', 'HEUT', 'PERFEKT', 'IS', 'SEHNSUCHT', 'MORG', 'HEUT', 'PERFEKT', 'IS', 'STEH', 'NEB', 'ZEIG', 'ECHT', 'LICHT', 'STADT', 'VERBRANNT', 'LASS', 'STRAND', 'KOMM', 'STEIG', 'LEUCHT', 'DIAMANT', 'YEH', 'ALL', 'SALZ', 'HAUT', 'AI', 'SEH', 'GESTOCH', 'SCHARF', 'TUT', 'FAST', 'AUG', 'WEH', 'ENDLICH', 'NOCH', 'NIE', 'GESEH', 'YEHI', 'WOLL', 'SEHNSUCHT', 'MORG', 'HEUT', 'PERFEKT', 'IS', 'SEHNSUCHT', 'MORG', 'HEUT', 'PERFEKT', 'IS', 'STEH', 'NEB', 'ZEIG', 'ECHT', 'LICHT', 'STADT', 'VERBRANNT', 'LASS', 'STRAND', 'LASS', 'STRAND', 'WEISST', 'WELCH', 'LASS', 'STRAND', 'YEAH', 'BRAUCHGRUND', 'BLEIB', 'AI', 'LASS', 'STRAND', 'WEISST', 'WELCH', 'BRAUCH', 'MAL', 'WIED', 'MEER', 'SAND', 'YEAH', 'BRAUCH', 'MAL', 'WIEDERGRUND', 'BLEIB', 'OUH', 'LASS',

In [72]:
print(df_Lied['TEXT'][0])

LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN ICH BRAUCH MAL WIEDER MEER UND SAND  YEAH BRAUCH MAL WIEDERGRUND ZU BLEIBEN  AI DEINE UHR FAELLT INS BLAU WIR SEHEN  WIE DIE ZEIT VERSINKT WIR SETZTEN LANGSAM AUF DAS WASSER SPIEGELGLATT UND BLINKT  WIR WOLLEN DIE SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS DU STEHST NEBEN MIR  ZEIGST MIR  DASS ES ECHT IST DIE LICHTER MEINER STADT SIND VERBRANNT LASS UNS ZUM STRAND    KOMM WIR STEIGEN AUS DU LEUCHTEST WIE AUS DIAMANT  YEHE MIT ALL DEM SALZ AUF DER HAUT  AI ICH SEH DICH GESTOCHEN SCHARF ES TUT FAST DEN AUGEN WEH UND ENDLICH BIST DU DA SO HABE ICH DICH NOCH NIE GESEHEN  YEHI  WIR WOLLEN DIE SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS DU STEHST NEBEN MIR  ZEIGST MIR  DASS ES ECHT IST DIE LICHTER MEINER STADT SIND VERBRANNT LASS UNS ZUM STRAND   LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN LASS UNS ZUM STRAND  YEAH ICH BRAUCHGRUND ZU BLEIB

Data Selection

In [75]:
df_Lied.drop(['TEXT_TEIL1','TEXT_TEIL2', 'TEXT_TEIL3', 'TEXT_TEIL4', 'SPRACHE_DEUTSCH'], axis=1, inplace=True)
df_Date = df_Chart_Position[['LIED_ID', 'DATUM_VON', 'DATUM_BIS', 'DAUER','JAHR', 'MONAT']]

#not used anymore:
#df_Chart_Position = df_Chart_Position.groupby('LIED_ID').agg({'POSITION':'mean','DAUER':'sum'}).reset_index()
#df_Chart_Position['POSITION'] = df_Chart_Position.POSITION.apply(lambda pos: round(pos))
#----

df_Lied.sort_values(by='ID', inplace=True)
df_Chart_Position.sort_values(by='LIED_ID', inplace=True)
df = pd.concat([df_Lied, df_Chart_Position], axis='columns')
df.drop('LIED_ID', axis=1, inplace=True)
print(df.head())

       ID       INTERPRET   TITEL  \
7219  408     Rumpelstilz  Kiosk    
7220  408     Rumpelstilz  Kiosk    
7763  425  Costa Cordalis  Anita    
7762  425  Costa Cordalis  Anita    
7761  425  Costa Cordalis  Anita    

                                                   TEXT  \
7219  ALSO ER SAMMLE FUER EINEN GUTEN ZWECK  SAGT DE...   
7220  ALSO ER SAMMLE FUER EINEN GUTEN ZWECK  SAGT DE...   
7763  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   
7762  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   
7761  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   

                                         processed_TEXT  POSITION  DATUM_VON  \
7219  [ALSO, SAMML, FUER, GUT, ZWECK, SAGT, FRITZ, S...        50 2009-03-06   
7220  [ALSO, SAMML, FUER, GUT, ZWECK, SAGT, FRITZ, S...         4 1984-10-22   
7763  [JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...        23 2003-08-25   
7762  [JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...        39 2003-07-28   
7761  [JVUIOUGIVTOH, F

In [77]:
df['ANZ_UNIQUE_WOERTER'] = list(len(set(word)) for word in df['processed_TEXT'])
df['MAX_WORT_WDH'] = [max(Counter(text).values()) for text in df['processed_TEXT']]
df['AVG_WORT_WDH'] = [mean(Counter(text).values()) for text in df['processed_TEXT']]
df['LAENGE_LAENGSTES_WORT'] = list(len(max(set(word), key=len)) for word in df['processed_TEXT'])

MAX_RANK = 50
df['RANK_SCORE'] = MAX_RANK - df['POSITION'] + 1

In [78]:
titel_List = set(df.TITEL.tolist())
for titel in titel_List:
    df.loc[df.TITEL == titel,'RANK_SCORE'] = round(mean(df.loc[df.TITEL == titel,'RANK_SCORE'].tolist()))

In [79]:
print(df['MAX_WORT_WDH'])

7219      6
7220      6
7763     16
7762     16
7761     16
         ..
22129    11
22014    42
22651    28
21906    67
22150    31
Name: MAX_WORT_WDH, Length: 22762, dtype: int64


profanity check

In [80]:
with open(RAW_DATA_PATH+'german-profanity-list.csv', newline='', encoding='UTF-8') as f:
    profanity_list = list(csv.reader(f))

profanity.load_words(flatten(profanity_list))

for text in df['processed_TEXT']:
    profanities = 0
    for word in text:
        if word.upper() in profanity_list:
            profanities +=1
            print(word)
            print(text)
            print(profanities)

Percentage of Stopwords

In [81]:
df['NUMBER_OF_STOPWORDS'] = df.TEXT.str.split().apply(lambda x: len(set(x) & set(stopwords_list)))
df['STOPWORD_PERCENTAGE'] = df.NUMBER_OF_STOPWORDS.apply(lambda row: round(row/len(df['TEXT']), ndigits=5))
print(df['STOPWORD_PERCENTAGE'].head(20))

7219    0.00211
7220    0.00211
7763    0.00189
7762    0.00189
7761    0.00189
7760    0.00189
7743    0.00189
7744    0.00189
7745    0.00189
7746    0.00189
7747    0.00189
7748    0.00189
7749    0.00189
7751    0.00189
7752    0.00189
7753    0.00189
7754    0.00189
7755    0.00189
7756    0.00189
7757    0.00189
Name: STOPWORD_PERCENTAGE, dtype: float64


Sentiment Analysis

In [82]:
df['SUBJECTIVITY'] = df.processed_TEXT.apply(lambda text: TextBlob(str(text)).sentiment.subjectivity)
df['POLARITY'] = df.processed_TEXT.apply(lambda text: TextBlob(str(text)).sentiment.polarity)

In [83]:
df.reset_index(inplace=True)
df.drop('index',axis=1, inplace=True)
print(df.head())

    ID       INTERPRET   TITEL  \
0  408     Rumpelstilz  Kiosk    
1  408     Rumpelstilz  Kiosk    
2  425  Costa Cordalis  Anita    
3  425  Costa Cordalis  Anita    
4  425  Costa Cordalis  Anita    

                                                TEXT  \
0  ALSO ER SAMMLE FUER EINEN GUTEN ZWECK  SAGT DE...   
1  ALSO ER SAMMLE FUER EINEN GUTEN ZWECK  SAGT DE...   
2  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   
3  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   
4  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   

                                      processed_TEXT  POSITION  DATUM_VON  \
0  [ALSO, SAMML, FUER, GUT, ZWECK, SAGT, FRITZ, S...        50 2009-03-06   
1  [ALSO, SAMML, FUER, GUT, ZWECK, SAGT, FRITZ, S...         4 1984-10-22   
2  [JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...        23 2003-08-25   
3  [JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...        39 2003-07-28   
4  [JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...        27 2003-07-28

LDA (BoW) - Title

Named Entity Recognition

In [32]:
test = [word for text in df_Lied['TEXT'] for word in nlp(text).ents]
print(test[0])

KeyboardInterrupt: 

Readability analysis

In [31]:
df.head(50)

,ID,INTERPRET,TITEL,Text,processed_Text,POSITION,DATUM_VON,DATUM_BIS,DAUER,Jahr,...,seasonal?,ANZ_UNIQUE_WOERTER,LAENGE_LAENGSTES_WORT,RANK_SCORE,Number_of_Stopwords,Stopword_Percentage,SUBJEKTIVITY,POLARITY,processed_Title,Title_length
0,408,Rumpelstilz,Kiosk,ALSO ER SAMMLE FUER EINEN GUTEN ZWECK SAGT DE...,"[ALSO, SAMMLE, FUER, GUT, ZWECK, SAGEN, FRITZ,...",50,2009-03-06,2009-03-12,6,2009,...,False,82,15,1,48,0.00211,0.40,0.1,[Kiosk],6
1,408,Rumpelstilz,Kiosk,ALSO ER SAMMLE FUER EINEN GUTEN ZWECK SAGT DE...,"[ALSO, SAMMLE, FUER, GUT, ZWECK, SAGEN, FRITZ,...",4,1984-10-22,1984-10-28,6,1984,...,False,82,15,47,48,0.00211,0.40,0.1,[Kiosk],6
2,425,Costa Cordalis,Anita,JVUIOUGIVTOH ICH FAND SIE IRGENDWO ALLEIN IN ...,"[JVUIOUGIVTOH, FINDEN, IRGENDWO, ALLEIN, MEXIK...",23,2003-08-25,2003-08-31,6,2003,...,False,74,12,28,43,0.00189,0.00,0.0,[Anita],6
3,425,Costa Cordalis,Anita,JVUIOUGIVTOH ICH FAND SIE IRGENDWO ALLEIN IN ...,"[JVUIOUGIVTOH, FINDEN, IRGENDWO, ALLEIN, MEXIK...",39,2003-07-28,2003-08-03,6,2003,...,False,74,12,12,43,0.00189,0.00,0.0,[Anita],6
4,425,Costa Cordalis,Anita,JVUIOUGIVTOH ICH FAND SIE IRGENDWO ALLEIN IN ...,"[JVUIOUGIVTOH, FINDEN, IRGENDWO, ALLEIN, MEXIK...",27,2003-07-28,2003-08-03,6,2003,...,False,74,12,24,43,0.00189,0.00,0.0,[Anita],6
5,425,Costa Cordalis,Anita,JVUIOUGIVTOH ICH FAND SIE IRGENDWO ALLEIN IN ...,"[JVUIOUGIVTOH, FINDEN, IRGENDWO, ALLEIN, MEXIK...",19,2003-07-28,2003-08-03,6,2003,...,False,74,12,32,43,0.00189,0.00,0.0,[Anita],6
6,425,Costa Cordalis,Anita,JVUIOUGIVTOH ICH FAND SIE IRGENDWO ALLEIN IN ...,"[JVUIOUGIVTOH, FINDEN, IRGENDWO, ALLEIN, MEXIK...",18,2016-07-08,2016-07-14,6,2016,...,False,74,12,33,43,0.00189,0.00,0.0,[Anita],6
7,425,Costa Cordalis,Anita,JVUIOUGIVTOH ICH FAND SIE IRGENDWO ALLEIN IN ...,"[JVUIOUGIVTOH, FINDEN, IRGENDWO, ALLEIN, MEXIK...",37,2016-07-08,2016-07-14,6,2016,...,False,74,12,14,43,0.00189,0.00,0.0,[Anita],6
8,425,Costa Cordalis,Anita,JVUIOUGIVTOH ICH FAND SIE IRGENDWO ALLEIN IN ...,"[JVUIOUGIVTOH, FINDEN, IRGENDWO, ALLEIN, MEXIK...",20,2016-09-16,2016-09-22,6,2016,...,False,74,12,31,43,0.00189,0.00,0.0,[Anita],6
9,425,Costa Cordalis,Anita,JVUIOUGIVTOH ICH FAND SIE IRGENDWO ALLEIN IN ...,"[JVUIOUGIVTOH, FINDEN, IRGENDWO, ALLEIN, MEXIK...",18,2016-09-23,2016-09-29,6,2016,...,False,74,12,33,43,0.00189,0.00,0.0,[Anita],6


Title Analysis

In [90]:
#lemmatize title
#df['processed_TITLE'] = df.TITEL.apply(lambda titel: ' '.join([x.lemma_ for x in nlp(titel)]))
#print(df['processed_TITLE'].head())

#tokenize title
tokenizer = RegexpTokenizer(r'\w+')
df['processed_TITLE'] = df.TITEL.apply(lambda titel: [word for word in tokenizer.tokenize(titel)])

#Stemming
df['processed_TITLE'] = df.processed_TITLE.apply(lambda titel: stemm_List_of_Words(titel))

#remove stopwords
df['processed_TITLE'] = df.processed_TITLE.apply(lambda titel: [word for word in titel if word.upper() not in stopwords_list])

#remove numbers
df['processed_TITLE'] = df.processed_TITLE.apply(lambda word_list : [re.sub('\w*\d\w*','', word) for word in word_list])

In [104]:
df['LENGTH_TITLE'] = df.TITEL.apply(lambda titel: len(titel))
list_words = [Counter(titel).items() for titel in df['processed_TITLE']]


df['TITEL_WORDS'] = list_words
df['TITEL_WORD_FREQ'] = counter.values()

TypeError: unhashable type: 'list'

Seasonal determination

In [101]:
df['SEASONAL?'] = np.nan
isSeasonal()
df['SEASONAL?']

0        False
1        False
2        False
3        False
4        False
         ...  
22757    False
22758    False
22759    False
22760    False
22761    False
Name: SEASONAL?, Length: 22762, dtype: object

Boosted Words -Analysis

<class 'pandas.core.series.Series'>


Language detection
Idea -> Mulitlangual classification boolean & Probability of German

In [5]:
#b = TextBlob("bonjour")
#b.detect_language()
detect_language('bonjour')


DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

In [4]:
def detect_language(text):
    """Detects the text's language."""
    translate_client = translate.Client()

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.detect_language(text)

    print("Text: {}".format(text))
    print("Confidence: {}".format(result["confidence"]))
    print("Language: {}".format(result["language"]))


In [13]:
authenticate_implicit_with_adc()

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

In [12]:
def authenticate_implicit_with_adc(project_id="subtle-seer-369823"):
    """
    When interacting with Google Cloud Client libraries, the library can auto-detect the
    credentials to use.

    // TODO(Developer):
    //  1. Before running this sample,
    //  set up ADC as described in https://cloud.google.com/docs/authentication/external/set-up-adc
    //  2. Replace the project variable.
    //  3. Make sure that the user account or service account that you are using
    //  has the required permissions. For this sample, you must have "storage.buckets.list".
    Args:
        project_id: The project id of your Google Cloud project.
    """

    # This snippet demonstrates how to list buckets.
    # *NOTE*: Replace the client created below with the client required for your application.
    # Note that the credentials are not specified when constructing the client.
    # Hence, the client library will look for credentials using ADC.
    storage_client = storage.Client(project=project_id)
    buckets = storage_client.list_buckets()
    print("Buckets:")
    for bucket in buckets:
        print(bucket.name)
    print("Listed all storage buckets.")

In [ ]:
from google.cloud import client
client.


Normalising

Export

In [102]:
df.to_csv('Data/processed/EDA.csv')
df_Date.to_csv('Data/processed/DATE.csv')